In [1]:
import pymongo
import pandas as pd
#from pandas import Series
import numpy as np
import soundfile as sf
#import matplotlib.pyplot as plt
#from collections import Counter
from bson.objectid import ObjectId
import json
import pprint
from pydub import AudioSegment
import urllib
import os
import requests
import ffmpy
import subprocess

/home/rrs432/miniconda2/envs/bandhub/lib/python2.7/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [6]:
client = pymongo.MongoClient('localhost',27017)
#client = pymongo.MongoClient('localhost',32768)
db = client.get_database('b=bandhub')

In [7]:
songCol = db.get_collection('songsStream')
vidCol = db.get_collection('mergedVideos')
trackCol = db.get_collection('tracksStream')
postCol = db.get_collection('posts')
[songCol.count(), vidCol.count(), trackCol.count(), postCol.count()]

[425706, 198169, 915582, 494867]

In [17]:
TwoTrackMixes = []
#zeroTrackMixes = 0
pubTracks = postCol.find({'songAccess':{'$exists': True}}, {'participantsInfo':1 ,'songAccess':1, '_id':1})
for docs in pubTracks:
    if(docs['songAccess'] == 1):
        if(len(docs['participantsInfo']['publishedTracks']) == 2):
            TwoTrackMixes.append(docs['_id'])
#pprint.pprint(len(TwoTrackMixes))

remix_test1 = TwoTrackMixes[0]
pprint.pprint(remix_test1)

ObjectId('546f05de4e5d9e60267e62da')


In [11]:
ThreeTrackMixes = []
masterSongIDs = []
#zeroTrackMixes = 0
pubTracks = postCol.find({'songAccess':{'$exists': True}}, {'participantsInfo':1 ,'songAccess':1, 'objectId':1})
for docs in pubTracks:
    if(docs['songAccess'] == 1):
        if(len(docs['participantsInfo']['publishedTracks']) == 3):
            #ThreeTrackMixes.append(docs['objectId'])
            ThreeTrackMixes.append(docs['_id'])
            masterSongIDs.append(docs['objectId'])
#pprint.pprint(len(TwoTrackMixes))

remix_test = ThreeTrackMixes[37]
songID = masterSongIDs[37]
pprint.pprint(songID)

ObjectId('524774cbbf9d3bda1e000019')


In [12]:
pub_t = []
rmt1 = postCol.find({'_id': remix_test})
count = 0
for docs in rmt1:
    if docs['songAccess'] == 1:
        #pprint.pprint(docs)
        #pprint.pprint(docs['participantsInfo']['publishedTracks'])
        count = len(docs['participantsInfo']['publishedTracks'])
        #pprint.pprint(docs['participantsInfo']['publishedTracks'])
        for i in range(count):
            pub_t.append(docs['participantsInfo']['publishedTracks'][i]['_id'])
        #pprint.pprint("Not public")
pprint.pprint(pub_t)

[ObjectId('52477674bf9d3bda1e00001c'),
 ObjectId('52477890bf9d3bda1e00001e'),
 ObjectId('53e7bbd6658a3a966ba074aa')]


In [45]:
songLookUp = songCol.find({'masterSongId' : songID})
pub_t_vol = []
ogg_paths = []
#pprint.pprint(songLookUp.count())
for docs in songLookUp:
    pprint.pprint(docs)
    for i in range(count):
        #pprint.pprint(docs['settings'][str(pub_t[i])]['volume'])
        #pprint.pprint(str(pub_t[i]))
        #pprint.pprint(docs['settings'][str(pub_t[i])]['volume'])
        pub_t_vol.append(docs['settings'][str(pub_t[i])]['volume'])
        #pprint.pprint(docs['settings'][str(pub_t[i])])
        if('effectsAudioUrl' in docs['settings'][str(pub_t[i])]): 
            ogg_paths.append(docs['settings'][str(pub_t[i])]['effectsAudioUrl'])
        else:
            gettrack = trackCol.find({'_id' : pub_t[i]})
            for tdocs in gettrack:
                ogg_paths.append(tdocs['audioChannels'][0]['fileUrl'])
                #pprint.pprint('No Effects Audio!')
    #print('found')
pprint.pprint(pub_t_vol)
pprint.pprint(ogg_paths)

{u'_id': ObjectId('524774cbbf9d3bda1e000019'),
 u'access': 1,
 u'communityOrder': 390,
 u'dateCreated': datetime.datetime(2013, 9, 29, 0, 31, 7),
 u'dateLastModified': datetime.datetime(2014, 3, 29, 7, 39, 26, 795000),
 u'description': u'',
 u'fbImage': u'http://bandhubmediastream.blob.core.windows.net/files/ct-524774cbbf9d3bda1e000019-2.png',
 u'fbImageTracks': [u'52477674bf9d3bda1e00001c',
                    u'52477890bf9d3bda1e00001e',
                    u'53e7bbd6658a3a966ba074aa'],
 u'hasPublishedTracks': True,
 u'hasRecordedTracks': True,
 u'isFeatured': False,
 u'masterSongId': ObjectId('524774cbbf9d3bda1e000019'),
 u'masterSongOwner': u'kari',
 u'migratedToAzure': True,
 u'mp4MergedMobileVideoUrl': u'http://bandhubmediastream.blob.core.windows.net/files/m-f-c1655efa-58ef-4e61-aac4-450d7d750a24.mp4',
 u'mp4MergedVideoUrl': u'http://bandhubmediastream.blob.core.windows.net/files/f-524774cbbf9d3bda1e000019.mp4',
 u'newMusicbrainzMetadataId': ObjectId('5696fa8a5a11481a904cb513'),

In [35]:
#ogg_paths = []
track_filenames = []
start_times = []
for i in range(count):
    gettrack = trackCol.find({'_id' : pub_t[i]})
    for docs in gettrack:
        #trackVideo = docs['videoFileUrl']
        #pprint.pprint(docs['audioChannels'][0]['fileUrl'])
        #ogg_paths.append(docs['audioChannels'][0]['fileUrl'])
        start_times.append(docs['startTimeValue'])
        #pprint.pprint(docs)
#pprint.pprint(ogg_path)
for file in ogg_paths:
    filePath = file.split('/')
    track_filenames.append(filePath[- 1])
    #print fileName
pprint.pprint(track_filenames)
pprint.pprint(start_times)

[u'e701ac78-3fae-4132-b94a-b2bd75f71bc9.ogg',
 u'91bbcaf8-2409-4eaf-93a9-a729ebe93274.ogg',
 u'2385b914-2fa2-4eab-bc0d-e9d0de9b2515.ogg']
[0, 0, 0]


In [36]:
path = '/scratch/rrs432/tracks'
#os.mkdir(path)
os.chdir(path)

for i in range(count):
    r = requests.get(ogg_paths[i])
    filename = ogg_paths[i].rpartition('/')[2]
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
print('Download successful')

Download successful


In [37]:
pointer = vidCol.find({'songId': songID})
for songdocs in pointer:
    premix = songdocs['oggMergedVideoUrl']
pprint.pprint(premix)

u'http://bandhubmediastream.blob.core.windows.net/files/f-91bced26-7277-49e8-ae1b-dc52c117d677.ogg'


In [38]:
path = '/scratch/rrs432/tracks'
#os.mkdir(path)
os.chdir(path)

pointer = vidCol.find({'songId': songID})
for songdocs in pointer:
    premix = songdocs['oggMergedVideoUrl']

r = requests.get(premix)
filename = premix.rpartition('/')[2]
with open(filename, 'wb') as fd:
    for chunk in r.iter_content(chunk_size=128):
        fd.write(chunk)
print('BandHub Mix Download successful')

BandHub Mix Download successful


In [39]:
ls

2385b914-2fa2-4eab-bc0d-e9d0de9b2515.ogg
91bbcaf8-2409-4eaf-93a9-a729ebe93274.ogg
e701ac78-3fae-4132-b94a-b2bd75f71bc9.ogg
f-91bced26-7277-49e8-ae1b-dc52c117d677.ogg
gconcr/
heyoh/


In [40]:
oggTracks = []
trackLens = []
samRates = []
for i in range(count):
    oggTrack, samRate = sf.read('/scratch/rrs432/tracks/'+track_filenames[i])
    samRates.append(samRate)
    trackLens.append(len(oggTrack))
    
    #pprint.pprint(np.shape(oggTracks[i])[1])
    if len(oggTrack.shape) == 2:
        oggTracks.append([np.mean(x) for x in oggTrack])
    else:
        oggTracks.append(oggTrack)
    
    pprint.pprint(np.shape(oggTracks[i]))
#pprint.pprint(oggTracks[0][0:100])
pprint.pprint(trackLens)
#pprint.pprint(len(oggTracks[2]))
#pprint.pprint(np.shape(samRates))

(9134400,)
(9137536,)
(9140288,)
[9134400, 9137536, 9140288]


In [183]:
# for i in range(count):
#     oggTrack, samRate = sf.read('/scratch/rrs432/tracks/'+track_filenames[i])
#     pprint.pprint(oggTrack[10000:10200])
#     break
# pprint.pprint(oggTracks[0][10000:10200])

In [41]:
if (min(samRates) == max(samRates)):
    mixsr = samRates[0] 
else:
    pprint.pprint('Warning: Sample rates unequal')
pprint.pprint(mixsr)

44100


In [42]:
masterLen = max(trackLens)
pprint.pprint(masterLen)

9140288


In [43]:
for i in range(count):
    pprint.pprint(len(oggTracks[i]))
    if(trackLens[i] != masterLen):
        #pprint.pprint(start_times[i])
        if(start_times[i] == 0):
            diff = np.zeros(masterLen - trackLens[i])
            #pprint.pprint(len(oggTracks[i]))
            oggTracks[i] = np.concatenate((oggTracks[i], diff), axis=0)
            #pprint.pprint(masterLen - trackLens[i])
            pprint.pprint(len(oggTracks[i]))
            #pprint.pprint(len(diff))
            if(len(oggTracks[i]) == masterLen):
                pprint.pprint('Successful')
            else:
                pprint.pprint('Try again')
        else:
            diff = np.zeros(start_times[i]-1)
            #pprint.pprint(len(diff))
            oggTracks[i] = np.concatenate((diff,oggTracks[i]), axis=0)
            diff = np.zeros(masterLen - len(oggTracks[i]))
            oggTracks[i] = np.concatenate((oggTracks[i], diff), axis=0)
            pprint.pprint(len(oggTracks[i]))
    else:
        pprint.pprint('Equal')

9134400
9140288
'Successful'
9137536
9140288
'Successful'
9140288
'Equal'


In [44]:
# path = '/scratch/rrs432/tracks'
# #os.mkdir(path)
# os.chdir(path)
#pprint.pprint(count)
mixOgg = 0
#pprint.pprint(len(pub_t_vol))
#pprint.pprint(len(oggTracks))
    #pprint.pprint(oggTracks[i])
for i in range(count):
    #pprint.pprint(pub_t_vol[i])
    #pprint.pprint(oggTracks[i])
    mixOgg += (pub_t_vol[i]*np.array(oggTracks[i]))
sf.write('mixed.wav', mixOgg / np.max(np.abs(mixOgg)), mixsr)

In [174]:
path = '/scratch/rrs432/tracks'
os.chdir(path)

In [84]:
!which ffmpeg

which: no ffmpeg in (/home/rrs432/miniconda2/envs/bandhub/bin:/share/apps/mongodb/3.4.10/bin:/home/rrs432/miniconda2/envs/bandhub/bin:/home/rrs432/miniconda2/bin:/usr/lib64/qt-3.3/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/ibutils/bin:/share/apps/local/bin:/share/apps/centos/7/bin:/opt/slurm/bin:/opt/dell/srvadmin/bin:/home/rrs432/.local/bin:/home/rrs432/bin)


In [81]:
subprocess.call(['ffmpeg', '-i', 'mixed.wav', '-codec:a', 'libvorbis', '-b:a', '232k', 'remixed.ogg'])

OSError: [Errno 2] No such file or directory

In [28]:
!which ffmpeg

which: no ffmpeg in (/home/rrs432/miniconda2/envs/bandhub/bin:/share/apps/mongodb/3.4.10/bin:/home/rrs432/miniconda2/envs/bandhub/bin:/home/rrs432/miniconda2/bin:/usr/lib64/qt-3.3/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/ibutils/bin:/share/apps/local/bin:/share/apps/centos/7/bin:/opt/slurm/bin:/opt/dell/srvadmin/bin:/home/rrs432/.local/bin:/home/rrs432/bin)


In [26]:
!ffmpeg -i 'mixed.wav' -codec:a libvorbis -b:a 232k remixed.ogg

/usr/bin/sh: ffmpeg: command not found


In [18]:
!ls

136dd451-e101-49a9-a7db-edef3f1ef098-0-0.ogg	     gconcr	premix.ogg
3258166c-a1ee-4f6a-9876-2d9325287a82-0.ogg	     heyoh	remixed.ogg
578feca1-696d-4d49-bb75-15dc63bb5822-0-0_surubi.ogg  mixed.wav


In [46]:
p, sr = sf.read('premixa.ogg')
r, sr = sf.read('remixed.ogg')
#pprint.pprint(np.shape(p)[1])
if np.shape(p)[1] == 2:
    p = p.sum(axis=1) / 2
pprint.pprint(np.shape(p))
pprint.pprint(np.shape(r))

(9143616,)
(9140288,)


In [47]:
pLen = len(p)
rLen = len(r)
if pLen < rLen:
    diff = np.zeros(rLen - pLen)
    p = np.concatenate((p,diff),axis = 0)
else:
    diff = np.zeros(pLen - rLen)
    r = np.concatenate((r,diff),axis = 0)
pprint.pprint(len(diff))
pprint.pprint(len(r))
pprint.pprint(len(p))

3328
9143616
9143616


In [119]:
diff = p-r
pprint.pprint(diff)

ValueError: operands could not be broadcast together with shapes (5734720,) (5731200,) 

In [53]:
np.allclose(p,r,atol=1e+01)

True

In [49]:
ls

8f127b81-5d41-1b61-c028-e769fbfe85c9.ogg
d4f684e0-1989-4abd-913a-5e78edebfe32-0-0.ogg
f-1827b0e1-9972-4c9f-bfe3-88db569ce857.ogg
mixed.ogg


In [5]:
data = pd.read_hdf('BandhubDatasetCopy.h5')

In [12]:
data.iloc[2]

trackId                                               54357096e18d72165e42ba85
songId                                                542fcd68e4251e5e66ec1cb2
masterOwner                                                             Brandx
trackOwner                                                              Brandx
artist                                                             Iron Maiden
title                                                      back in the village
views                                                                      800
instrument                                                                 NaN
contentTags                                                               null
audioURL                     http://bandhubwebmedia4.blob.core.windows.net/...
cleanProcessedAudioURL       http://bandhubwebmedia5.blob.core.windows.net/...
processedAudioURL                                                          NaN
startTime                                           

In [ ]:
path = '/home/rrs432/'
#os.mkdir(path)
os.chdir(path)